In [1]:
from bs4 import BeautifulSoup
import boto3

In [2]:
def read_in_file(bucket, path):
    s3_client = boto3.client('s3')
    s3_response_object = s3_client.get_object(Bucket=bucket, Key=path)
    html_str = s3_response_object['Body'].read()
    return html_str.decode('utf-8')


# list files of s3 bucket
def list_files(bucket, path):
    s3_client = boto3.client('s3')
    s3_response_object = s3_client.list_objects(Bucket=bucket, Prefix=path)
    return s3_response_object['Contents']

In [3]:
bucket = 'scrape-projects'

In [154]:
files = list_files(bucket, path='colossus-transcripts/html/')

In [155]:
path = files[-3]['Key']

In [195]:
path

'colossus-transcripts/html/Investing-in-Commercial-Real-Estate.html'

In [196]:
html = read_in_file(bucket, path)

In [247]:
soup = BeautifulSoup(html, 'html.parser')

In [172]:
conversation = []
names = []
for item in soup.find_all('p'):
    if ':' in item.text and '[' in item.text:
        s = item.text.split(":")
        name = s[0]
        time = re.search(r'\[\d{2}:\d{2}:\d{2}\]', item.text).group()
        names.append(name)
#         print(name, time)
    conversation.extend([item.text])

In [201]:
from collections import OrderedDict

In [206]:
interviewers = list(OrderedDict.fromkeys(names).keys())

In [179]:
convo = " ".join(conversation)

In [239]:
import re
string = f"(?:{interviewers[0]}|{interviewers[1]}):\s*\[\d{{2}}:\d{{2}}:\d{{2}}\]\s*"
# paragraphs = re.split(r'(?:Patrick|Bill):\s*\[\d{2}:\d{2}:\d{2}\]\s*', convo)
paragraphs = re.split(string, convo)

In [241]:
conversation[0]

"Patrick: [00:02:24] My guest today is Bill Lenehan. Bill is the CEO of Four Corners Property Trust, a listed REIT and one of the leading owners of restaurant real estate in the U.S. Their portfolio is made up of 982 properties across 47 states. Real estate is something that most of us own whether as an investment or as a home. And Bill's insight into the asset class, at this particular moment in time, is fascinating to hear. Please enjoy my conversation with Bill Lenehan."

In [242]:
transcript = []
for n, p in zip(names, paragraphs[1:]):
    d = {'name': n, 'response': p}
    transcript.append(d)

In [243]:
transcript[1:]

[{'name': 'Patrick',
  'response': "So Bill, this is going to be a really neat opportunity to talk with someone that I probably learned the most from about the asset class of real estate writ large, not my background or area of expertise. But I think it's interesting to everyone because we're surrounded by it all the time. We all buy houses or many people do. We work in offices, we visit malls, we do all these things. An interesting orienting place to begin is actually with something topical, even though I think a lot of what we'll talk about today is a bit more timeless just because of the nature of the environment that we're talking in, which is November, what date is it today? Like 18th 2022. Rates are the obvious place, I think, to begin because everyone's familiarity with the situation of rates have gone up a lot. Mortgage rates have gone up a lot. So the housing market hasn't really crashed so much as it's sort of frozen. Nobody's moving, no one wants to go from a 2% to an 8% mor

In [255]:
def parse(soup):
    conversation = []
    names = []
    for item in soup.find_all('p'):
        if ':' in item.text and '[' in item.text:
            s = item.text.split(":")
            name = s[0]
            time = re.search(r'\[\d{2}:\d{2}:\d{2}\]', item.text).group()
            names.append(name)
        conversation.extend([item.text])
    interviewers = list(OrderedDict.fromkeys(names).keys())
    conversation = " ".join(conversation)
    return {'interviewers': interviewers, 'conversation': conversation}

def process(interviewers, conversation):
    string = f"(?:{interviewers[0]}|{interviewers[1]}):\s*\[\d{{2}}:\d{{2}}:\d{{2}}\]\s*"
    paragraphs = re.split(string, conversation)
    transcript = []
    for n, p in zip(names[1:], paragraphs[2:]):
        d = {'name': n, 'response': p}
        transcript.append(d)
        
    return transcript

In [256]:
d = parse(soup)

In [258]:
processed_transcript = process(d['interviewers'], d['conversation'])

In [ ]:
wr.s3.to